## Hiperparâmetros com Bayesian Optimization

O ajuste de hiperpârametros em um modelo de **Machine Learning** é extremamente importante, já que pode elevar substancialmente o desempenho do modelo em aderência aos dados.
É muito comum ver em tutoriais de Machine Learning o uso do **Grid Search** para ajustar os hiperparâmetros, entranto há um problema de custo de processamento alto já que o Grid Search irá executar vários modelos e testar todo o intervalo de hiperparâmetros definido pelo usuário.
Em contra partida o **Bayesian
